In [1]:
import yfinance as yf
#import investpy as py
import pandas as pd
from datetime import date, timedelta
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType, DateType, TimestampType, FloatType
from pyspark.sql.functions import udf, col, lit, year, month, date_trunc

#### call lib

In [2]:
%run ./work/lib.ipynb

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9f96d11b-057e-4247-8d53-20faf1ecee31;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/1.0.0/delta-core_2.12-1.0.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;1.0.0!delta-core_2.12.jar (851ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4/4.7/antlr4-4.7.jar ...
	[SUCCESSFUL ] org.antlr#antlr4;4.7!antlr4.jar (216ms)
downloading https:

##### base date

In [3]:
load_years = 15
date_today = date.today()
base_date = date_today - timedelta(days= 30 * 12 * load_years)
base_date = date(base_date.year, 1, 1)

##### stock list

In [4]:
stocks_df = \
    spark \
        .read \
        .format('csv') \
        .option('header', 'true') \
        .option('delimiter', ';') \
        .load('{}/extract/stock_list.csv'.format(LAKE_HOME))

stock_list = \
    stocks_df \
        .rdd \
        .map(lambda x: x[0]) \
        .collect()

#### dividends

In [5]:
def get_dividends(tickers: list):
    import pandas as pd
    df = pd.DataFrame()

    schema = StructType([
        StructField('date', TimestampType(), True),
        StructField('dividend', FloatType(), True),
        StructField('ticker', StringType(), True),
    ])

    for ticker in tickers:
       
        try:
            download = pd.DataFrame()
            ticker_request = ticker + '.SA'
            print(f'Buscando info: {ticker}')
            download['dividends'] = yf.Ticker(ticker_request).dividends
            download['ticker'] = ticker
            df = df.append(download)
        except:
            print(f'Error {ticker}...')
            continue

    df.rename_axis('date', inplace=True)
    df.reset_index(inplace=True)

    table = spark.createDataFrame(df, schema)
    
    return table

In [6]:
df = get_dividends(tickers=stock_list)

Buscando info: ABEV3
Buscando info: AMER3
Buscando info: AURA33
Buscando info: B3SA3
Buscando info: BBDC4
Buscando info: BBAS3
Buscando info: BRKM5
Buscando info: BRFS3
Buscando info: CXSE3
Buscando info: ELET6
Buscando info: CSNA3
Buscando info: SBSP3
Buscando info: SAPR11
Buscando info: CMIG4
Buscando info: CPLE6
Buscando info: TRPL4
Buscando info: DXCO3
Buscando info: EGIE3
Buscando info: FLRY3
Buscando info: GGBR4
Buscando info: IRBR3
Buscando info: ITUB4
Buscando info: ITSA4
Buscando info: JBSS3
Buscando info: KLBN4
Buscando info: MGLU3
Buscando info: POMO4
Buscando info: PETR4
Buscando info: PRIO3
Buscando info: SHUL4
Buscando info: SQIA3
Buscando info: SLCE3
Buscando info: VIVT3
Buscando info: TAEE11
Buscando info: TUPY3
Buscando info: UNIP6
Buscando info: USIM5
Buscando info: VALE3
Buscando info: VIIA3
Buscando info: WEGE3


In [7]:
fact_dividends = \
    df \
        .withColumn('year', year(col('date'))) \
        .withColumn('month', (date_trunc('month', col('date'))).cast('date')) \
        .select(
            col('date').cast('date').alias('date'),
            col('year'),
            col('month'),
            col('ticker'),
            col('dividend')
        )

##### Write to DW

In [8]:
df = fact_dividends

write_to_dw(df, 'fact_dividend')